# Add sidewalk widths to basic pedestrian network

In [1]:
# Select where to run notebook: "azure" or "local"
my_run = "azure"

In [2]:
import set_path

import math

import numpy as np
import pandas as pd

import shapely.ops as so
import shapely.geometry as sg
import geopandas as gpd
from geopandas import GeoDataFrame
from centerline.geometry import Centerline

import networkx as nx
import momepy

from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

import poly_utils
import bgt_utils

import matplotlib.pyplot as plt
import folium

import settings as st
if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf

/mnt/batch/tasks/shared/LS_root/mounts/clusters/tutorials-compute/code/Users/shayla.jansen/Accessible_Route_Planning/venv_rp/lib/python3.8/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_36438/726904891.py:10: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have P

In [4]:
os.system('sudo blobfuse /home/azureuser/cloudfiles/code/blobfuse/sidewalk --tmp-path=/mnt/resource/blobfusetmp --config-file=/home/azureuser/cloudfiles/code/blobfuse/fuse_connection_sidewalk.cfg -o attr_timeout=3600 -o entry_timeout=3600 -o negative_timeout=3600 -o allow_other -o nonempty')

0

## Import basic network data

In [5]:
# Get basic pedestrian network and sidewalks
df_bgt_nw = gpd.read_file(cf.output_basic_network)
df_bgt_sw = gpd.read_file(cf.output_sidewalks_basic_network)

In [6]:
df_bgt_nw.rename(columns={'geometry': 'centerlines'}, inplace=True)
df_bgt_exp = pd.merge(df_bgt_nw, df_bgt_sw, on='sidewalk_id', how='left')
df_bgt_exp = df_bgt_exp.set_geometry('centerlines')
df_bgt_exp.head(3)

,cl_id,length,sidewalk_id,centerlines,area,geometry
0,0,0.856342,0,"LINESTRING (114335.503 485523.413, 114335.766 ...",634.10523,"POLYGON ((114335.35700 485525.03598, 114335.46..."
1,1,1.990000,0,"LINESTRING (114336.060 485524.041, 114336.115 ...",634.10523,"POLYGON ((114335.35700 485525.03598, 114335.46..."
2,1,1.990000,0,"LINESTRING (114336.871 485525.837, 114336.893 ...",634.10523,"POLYGON ((114335.35700 485525.03598, 114335.46..."


## Create relevant variable - full width

### Get width and color

In [7]:
df_bgt_exp[['avg_width', 'min_width']] = df_bgt_exp.progress_apply(
    lambda row: poly_utils.get_avg_width_cl(row.geometry, row.centerlines, 
                                            st.width_resolution, st.width_precision), axis=1)

  0%|          | 0/55609 [00:00<?, ?it/s]

/mnt/batch/tasks/shared/LS_root/mounts/clusters/tutorials-compute/code/Users/shayla.jansen/Accessible_Route_Planning/src/poly_utils.py:107: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if line.type == 'MultiLineString':
/mnt/batch/tasks/shared/LS_root/mounts/clusters/tutorials-compute/code/Users/shayla.jansen/Accessible_Route_Planning/src/poly_utils.py:115: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if line.type == 'LineString':


In [8]:
conditions = [
    (df_bgt_exp['min_width'] < st.width_1),
    (df_bgt_exp['min_width'] >= st.width_1) & (df_bgt_exp['min_width'] < st.width_2),
    (df_bgt_exp['min_width'] >= st.width_2) & (df_bgt_exp['min_width'] < st.width_3),
    (df_bgt_exp['min_width'] >= st.width_3) & (df_bgt_exp['min_width'] < st.width_4),
    (df_bgt_exp['min_width'] >= st.width_4) & (df_bgt_exp['min_width'] < st.width_5),
    (df_bgt_exp['min_width'] >= st.width_5) & (df_bgt_exp['min_width'] < st.width_6),
    (df_bgt_exp['min_width'] >= st.width_6)
]

values_float = [st.min_path_width, st.width_1, st.width_2, st.width_3, st.width_4, st.width_5, st.width_6]
values_indication = ['<' + str(st.width_1) + 'm', 
                     str(st.width_1) + '-' + str(st.width_2) + 'm', 
                     str(st.width_2) + '-' + str(st.width_3) + 'm', 
                     str(st.width_3) + '-' + str(st.width_5) + 'm', 
                     str(st.width_4) + '-' + str(st.width_5) + 'm', 
                     str(st.width_5) + '-' + str(st.width_6) + 'm', 
                     '>' + str(st.width_6) + 'm']

In [9]:
df_bgt_exp['full_width_float'] = np.select(conditions, values_float)
df_bgt_exp['full_width'] = np.select(conditions, values_indication)
df_bgt_exp['full_width'].value_counts()

full_width
>1.6m       44546
0.6-0.8m     1997
1.4-1.6m     1963
0.8-1.0m     1954
<0.6m        1891
1.0-1.4m     1757
1.2-1.4m     1501
Name: count, dtype: int64

### Remove too narrow paths and short-ends

In [10]:
# Apply minimal path width on BGT centerlines
print(df_bgt_exp.shape)
df_bgt_exp = df_bgt_exp[df_bgt_exp['min_width'] > st.min_path_width].reset_index(drop=True)
print(df_bgt_exp.shape)

(55609, 10)
(54956, 10)


In [11]:
# Remove short lines
df_bgt_exp = df_bgt_exp.rename(columns={'geometry':'geometry_sidewalks', 'centerlines':'geometry'}) 
mls_per_id = poly_utils.create_mls_per_sidewalk(df_bgt_exp, crs=st.CRS)
mls_per_id['geometry'] = mls_per_id['geometry'].progress_apply(
    lambda x: poly_utils.remove_short_lines(x, st.min_se_length_fw))

  0%|          | 0/354 [00:00<?, ?it/s]

/mnt/batch/tasks/shared/LS_root/mounts/clusters/tutorials-compute/code/Users/shayla.jansen/Accessible_Route_Planning/src/poly_utils.py:49: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if line.type == 'MultiLineString':


In [12]:
# Apply selection of longer lines to original dataframe
long_segments_df = gpd.GeoDataFrame(mls_per_id.geometry.explode())
df_bgt_exp = df_bgt_exp.merge(long_segments_df, how='inner')
df_bgt_exp.shape

/tmp/ipykernel_36438/3247731847.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  long_segments_df = gpd.GeoDataFrame(mls_per_id.geometry.explode())


(48645, 10)

In [13]:
df_bgt_exp.head()

,cl_id,length,sidewalk_id,geometry,area,geometry_sidewalks,avg_width,min_width,full_width_float,full_width
0,1,1.99,0,"LINESTRING (114336.060 485524.041, 114336.115 ...",634.10523,"POLYGON ((114335.35700 485525.03598, 114335.46...",1.9,1.7,1.6,>1.6m
1,1,1.99,0,"LINESTRING (114336.871 485525.837, 114336.893 ...",634.10523,"POLYGON ((114335.35700 485525.03598, 114335.46...",1.7,1.6,1.6,>1.6m
2,1,1.99,0,"LINESTRING (114338.513 485526.844, 114338.543 ...",634.10523,"POLYGON ((114335.35700 485525.03598, 114335.46...",1.6,1.6,1.6,>1.6m
3,1,1.99,0,"LINESTRING (114340.391 485527.449, 114340.447 ...",634.10523,"POLYGON ((114335.35700 485525.03598, 114335.46...",1.6,1.6,1.6,>1.6m
4,1,1.99,0,"LINESTRING (114342.269 485528.055, 114342.351 ...",634.10523,"POLYGON ((114335.35700 485525.03598, 114335.46...",1.6,1.6,1.6,>1.6m


## Import segments data

In [14]:
# Read lines with widths (calculated in notebook 5 of https://github.com/Amsterdam-AI-Team/Urban_PointCloud_Sidewalk_Width)
df_segments_full = gpd.read_file(cf.segments_file, crs=st.CRS)

### Selection of segments within pilot area

In [15]:
# Import areas
df_areas = gpd.read_file(cf.output_pilot_area)

# Only keep BGT data within pilot areas
df_segments = df_segments_full.sjoin(df_areas, how='inner', predicate='within')  # note: only sidewalk polygons fully inside area are included

del df_segments_full

## Create relevant variable - obstacle-free width

### Remove too narrow paths and short-ends

In [16]:
# Apply minimal path width
df_segments_wide = df_segments[df_segments['min_width'] > st.min_path_width].reset_index(drop=True)
print(df_segments.shape)
print(df_segments_wide.shape)

(107301, 8)
(99544, 8)


In [17]:
# Remove short lines (this takes a while)
mls_per_id = poly_utils.create_mls_per_sidewalk(df_segments_wide, crs=st.CRS)
mls_per_id['geometry'] = mls_per_id['geometry'].progress_apply(poly_utils.remove_short_lines)

  0%|          | 0/394 [00:00<?, ?it/s]

/mnt/batch/tasks/shared/LS_root/mounts/clusters/tutorials-compute/code/Users/shayla.jansen/Accessible_Route_Planning/src/poly_utils.py:49: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if line.type == 'MultiLineString':


In [18]:
# Apply selection of longer lines to original dataframe
long_segments_df = gpd.GeoDataFrame(mls_per_id.geometry.explode())
df_segments_wide = df_segments_wide.merge(long_segments_df, how='inner')
df_segments_wide.shape

/tmp/ipykernel_36438/2035344241.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  long_segments_df = gpd.GeoDataFrame(mls_per_id.geometry.explode())


(91574, 8)

### Apply width factor

In [19]:
# Add width factor, for calculating the weights of the paths later
conditions = [
    (df_segments_wide['min_width'] < st.width_1),
    (df_segments_wide['min_width'] >= st.width_1) & (df_segments_wide['min_width'] < st.width_2),
    (df_segments_wide['min_width'] >= st.width_2) & (df_segments_wide['min_width'] < st.width_3),
    (df_segments_wide['min_width'] >= st.width_3) & (df_segments_wide['min_width'] < st.width_4),
    (df_segments_wide['min_width'] >= st.width_4) & (df_segments_wide['min_width'] < st.width_5),
    (df_segments_wide['min_width'] >= st.width_5) & (df_segments_wide['min_width'] < st.width_6),
    (df_segments_wide['min_width'] >= st.width_6)
]
values_factor = [1000000000000, 10000000000, 100000000, 1000000, 10000, 100, 1]
df_segments_wide['min_width_factor'] = np.select(conditions, values_factor).astype('int64')

### Take point cloud coverage into account

In [20]:
df_segments_wide['min_width_factor'][df_segments_wide['pc_coverage'] == False] = 100000000000001

/tmp/ipykernel_36438/1379915908.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_segments_wide['min_width_factor'][df_segments_wide['pc_coverage'] == False] = 100000000000001


In [21]:
df_segments_wide['min_width_factor'].value_counts()

min_width_factor
1                70458
100               4132
1000000           4033
10000             3835
100000000         3783
10000000000       3716
1000000000000     1617
Name: count, dtype: int64

### Do network calculation

In [22]:
df_bgt_exp['route_weight'] = np.nan
print('start doing network calculation on ' + str(len(df_bgt_sw['sidewalk_id'])) + ' rows')

start doing network calculation on 354 rows


In [23]:
# Create final dataframe   # TODO .loc[j, 'route_weight'] = ... everywhere
final_df = pd.DataFrame()

for i in range(len(df_bgt_sw['sidewalk_id'])):
    print(i)
      
    # Get sidewalk polygon for this centerline 
    my_sidewalk = df_bgt_sw['geometry'].values[i]   
    
    # Create graph for all paths withing this sidewalk polygon
    df_sidewalk = df_segments_wide[df_segments_wide['geometry'].within(my_sidewalk)].reset_index(drop=True)
    G = momepy.gdf_to_nx(df_sidewalk, approach="primal", multigraph=True)
    
    # Create dataframe with linestrings of centerline
    centerline_df = df_bgt_exp[df_bgt_exp['sidewalk_id'] == i].reset_index(drop=True)
    
    for j in range(len(centerline_df['geometry'])):  
        
        # Get line
        my_line = centerline_df.iloc[[j]]['geometry'].values[0]
        
        if len(list(G.nodes)) > 0:
            # Check if my_line has start and end (not a ring)
            if len(my_line.boundary) == 0:
                print('no route calculated for line (j)', j, 'in sidewalk (i)', i, '(ring)')
                centerline_df['route_weight'][j] = 0
            else:
                # Get origin and destination location
                origin_point, dest_point = my_line.boundary
                origin_node_loc = so.nearest_points(origin_point, sg.MultiPoint(list(G.nodes)))[1]
                dest_node_loc = so.nearest_points(dest_point, sg.MultiPoint(list(G.nodes)))[1]

                # Get origin and destination node
                origin_node = (origin_node_loc.x, origin_node_loc.y)
                dest_node = (dest_node_loc.x, dest_node_loc.y)   

                # Get weight of optimal route in graph
                try:
                    route_weight = nx.shortest_path_length(G, origin_node, dest_node,
                                                           weight='min_width_factor')
                    if (origin_point.distance(origin_node_loc) < st.max_dist) and (dest_point.distance(dest_node_loc) < st.max_dist):
                        centerline_df['route_weight'][j] = route_weight
                    else:
                        print('origin and/or destination node too far from line start/end for line (j)', j, 'in sidewalk (i)', i) 
                        centerline_df['route_weight'][j] = np.nan
                except nx.NetworkXNoPath:
                    print('no route found for line (j)', j, 'in sidewalk (i)', i)
                    centerline_df['route_weight'][j] = 100000000000000
        else:
            print('network has zero nodes')

    # Append data to final dataframe
    final_df = pd.concat([final_df, centerline_df])
final_df = final_df.reset_index()

0


TypeError: object of type 'MultiPoint' has no len()

In [ ]:
#del df_bgt_exp

In [ ]:
# Get final width of the routes
final_df['obstacle_free_width_float'] = final_df.progress_apply(    
    lambda row: poly_utils.get_route_width(row.route_weight), axis=1)

## Post-process 

In [ ]:
final_df = final_df.set_geometry('geometry')

In [ ]:
final_df['obstacle_free_width_float'].value_counts(dropna=False)

### Fix incorrect labeling of obstacle-free width (> full width)

In [ ]:
# Start a column to track any adjustments we make to the obstacle-free width
final_df['width_fill'] = np.where(final_df['obstacle_free_width_float'] > final_df['full_width_float'], 1, 0)

# If obstacle free width float is bigger than full width, set obstacle free width float to full width
final_df['obstacle_free_width_float'] = np.where(final_df['obstacle_free_width_float'] > final_df['full_width_float'], final_df['full_width_float'], final_df['obstacle_free_width_float'])

### Fix widths of 'inritten' 
These are not properly accounted for, as they were not included in the previous steps of the widths pipeline. 
Here, we assume that there are no obstacles on 'inritten'. Therefore, we can set the obstacle-free widths of these to the full widths.  
This doesn't resolve the issue for 'voetgangersgebied' & 'woonerf', but here the above assumption would be too much of a stretch. (luckily there are only 2 'voetgangersgebieden' and 0 'woonerven' in our pilot area. 

In [ ]:
## TODO remove
final_df.to_file('../data/widths_network_intermediate_inrit.gpkg', driver='GPKG') 

In [ ]:
## TODO


### Fix unknown widths

In [ ]:
# Fill unknown widths with minimum width of neighbors
for i, row in final_df.iterrows():
    if math.isnan(row['obstacle_free_width_float']):
        my_edge = row['geometry']
        my_dist = final_df['geometry'].distance(my_edge)
        my_nb_ids = my_dist.loc[my_dist == 0].index.tolist()
        new_width = min(final_df['obstacle_free_width_float'].iloc[my_nb_ids])
        if math.isnan(new_width):
            # If no neighbors with known width are found, fill with lowest width category
            final_df.loc[i, 'obstacle_free_width_float'] = st.min_path_width
            final_df.loc[i, 'width_fill'] = 3
        else:
            final_df.loc[i, 'obstacle_free_width_float'] = new_width
            final_df.loc[i, 'width_fill'] = 2


In [ ]:
final_df['obstacle_free_width_float'].value_counts(dropna=False)

In [ ]:
final_df['width_fill'].value_counts(dropna=False)

### Set final indication

In [ ]:
# Get dataframe with only valid lines
final_df_select = final_df[final_df['obstacle_free_width_float'] != 0]
final_df_select = final_df_select.reset_index(drop=True)

In [ ]:
# Add meter indication
conditions = [
    (final_df_select['obstacle_free_width_float'] == st.min_path_width),
    (final_df_select['obstacle_free_width_float'] == st.width_1),   
    (final_df_select['obstacle_free_width_float'] == st.width_2),
    (final_df_select['obstacle_free_width_float'] == st.width_3),
    (final_df_select['obstacle_free_width_float'] == st.width_4),
    (final_df_select['obstacle_free_width_float'] == st.width_5),  
    (final_df_select['obstacle_free_width_float'] == st.width_6),  
    (final_df_select['obstacle_free_width_float'] == np.nan)
]
values_indication = ['<' + str(st.width_1) + 'm', 
                     str(st.width_1) + '-' + str(st.width_2) + 'm', 
                     str(st.width_2) + '-' + str(st.width_3) + 'm', 
                     str(st.width_3) + '-' + str(st.width_4) + 'm', 
                     str(st.width_4) + '-' + str(st.width_5) + 'm', 
                     str(st.width_5) + '-' + str(st.width_6) + 'm', 
                     '>' + str(st.width_6) + 'm',
                    'unknown']
final_df_select['obstacle_free_width'] = np.select(conditions, values_indication)

### Adjust crs

In [ ]:
final_df_select = final_df_select.set_crs(st.CRS)  # TODO might be removed?
print('finished post-processing')

## Merge with basic network

In [ ]:
# Select relevant columns for merging
final_df_select = final_df_select[['geometry', 'obstacle_free_width', 'obstacle_free_width_float', 'width_fill', 
                                   'full_width', 'full_width_float']]

In [ ]:
# Do inner join with original network
df_combined = pd.merge(df_bgt_nw, final_df_select, how='inner', left_on='centerlines', right_on='geometry')

## Store final output

In [ ]:
df_store = df_combined[['cl_id', 'centerlines', 'length', 'sidewalk_id', 'obstacle_free_width', 'obstacle_free_width_float', 'width_fill']]
df_store = df_store.set_geometry('centerlines').set_crs(st.CRS)
df_store.head()

In [ ]:
df_store.to_file(cf.output_file_widths, driver='GPKG') 
print('stored final data')

## Visualize

In [ ]:
# df_store = gpd.read_file(cf.output_file_widths)

In [ ]:
# Set center of map
area_coords = [52.375664816888225, 4.8632280955697995]

# Set to map coordinate reference system
df_store_show = df_store.to_crs("EPSG:4326")

# Create basic map
my_map = folium.Map(
    location=area_coords,
    tiles='cartodbpositron',
    min_zoom=10, max_zoom=20, zoom_start=13,
    zoom_control=True, control_scale=True, control=False
    )

# Add network with widths
folium.GeoJson(data=df_store_show[df_store_show['obstacle_free_width_float'] == st.min_path_width], style_function=lambda x: {"color": "darkred"}).add_to(my_map)
folium.GeoJson(data=df_store_show[df_store_show['obstacle_free_width_float'] == st.width_1], style_function=lambda x: {"color": "red"}).add_to(my_map)
folium.GeoJson(data=df_store_show[df_store_show['obstacle_free_width_float'] == st.width_2], style_function=lambda x: {"color": "orange"}).add_to(my_map)
folium.GeoJson(data=df_store_show[df_store_show['obstacle_free_width_float'] == st.width_3], style_function=lambda x: {"color": "yellow"}).add_to(my_map)
folium.GeoJson(data=df_store_show[df_store_show['obstacle_free_width_float'] == st.width_4], style_function=lambda x: {"color": "greenyellow"}).add_to(my_map)
folium.GeoJson(data=df_store_show[df_store_show['obstacle_free_width_float'] == st.width_5], style_function=lambda x: {"color": "limegreen"}).add_to(my_map)
folium.GeoJson(data=df_store_show[df_store_show['obstacle_free_width_float'] == st.width_6], style_function=lambda x: {"color": "green"}).add_to(my_map)

my_map 

In [ ]:
# Store map
my_map.save(cf.network_map_widths)